In [1]:
import pandas as pd

In [2]:
import re

In [3]:
data = pd.read_csv('../native_ad_data.csv')

In [4]:
data.head()

,_id,headline,link,img,provider,source,img_file,date,final_link
0,ObjectId(58fc4a639e1e2a693fe54747),\nWhy People At NBC Are Reportedly Worried Abo...,http://www.zergnet.com/i/1597590/50261/0/0/139...,http://img5.zergnet.com/1597590_300.jpg,zergnet,http://tmz.com,76f999e904074946360b8116ed46841d6c26d2df.jpg,2017-04-23T06:30:59.796Z,http://www.zergnet.com/news/1597590/why-people...
1,ObjectId(58fc4a639e1e2a693fe54748),\nMagic Johnson Opens Up About His Son EJ Comi...,http://www.zergnet.com/i/1642211/50261/0/0/139...,http://img4.zergnet.com/1642211_300.jpg,zergnet,http://tmz.com,fab1d021d6f6a57c778988a90aa7803375e2550a.jpg,2017-04-23T06:30:59.835Z,http://www.zergnet.com/news/1642211/magic-john...
2,ObjectId(58fc4a639e1e2a693fe54749),\nAnthony Bourdain Relieved to No Longer Prete...,http://www.zergnet.com/i/1597599/50261/0/0/139...,http://img4.zergnet.com/1597599_300.jpg,zergnet,http://tmz.com,db6f8f33bfca04150222fb771cfc86fd572f6e9b.jpg,2017-04-23T06:30:59.889Z,http://www.zergnet.com/news/1597599/anthony-bo...
3,ObjectId(58fc4a639e1e2a693fe5474a),\nThe Real Reason Demi Moore & Ashton Kutcher ...,http://www.zergnet.com/i/1611238/50261/0/0/139...,http://img5.zergnet.com/1611238_300.jpg,zergnet,http://tmz.com,ba5f7e095f8d048dfbe36997a122205ed8847bcd.jpg,2017-04-23T06:30:59.943Z,http://www.zergnet.com/news/1611238/the-real-r...
4,ObjectId(58fc4a639e1e2a693fe5474b),\nCarmelo and La La Anthony's Prenup Will Cont...,http://www.zergnet.com/io/1646212/50262/0/0/13...,http://img1.zergnet.com/1646212_300.jpg,zergnet,http://tmz.com,2c2eded673d1577bb7f5e139867bf7d08289eeae.jpg,2017-04-23T06:31:00.003Z,http://www.tmz.com/2017/04/20/carmelo-la-la-an...


The one's from zergnet all have some newlines we need to get rid of and they appear to have concatenated the headline with the provider. So let's clean those up.

In [5]:
data['headline'] = data['headline'].apply(lambda x: re.sub('(?<=[a-z])\.?([A-Z](.*))' , '', x.strip()))
data.head()

,_id,headline,link,img,provider,source,img_file,date,final_link
0,ObjectId(58fc4a639e1e2a693fe54747),Why People At NBC Are Reportedly Worried About...,http://www.zergnet.com/i/1597590/50261/0/0/139...,http://img5.zergnet.com/1597590_300.jpg,zergnet,http://tmz.com,76f999e904074946360b8116ed46841d6c26d2df.jpg,2017-04-23T06:30:59.796Z,http://www.zergnet.com/news/1597590/why-people...
1,ObjectId(58fc4a639e1e2a693fe54748),Magic Johnson Opens Up About His Son EJ Coming...,http://www.zergnet.com/i/1642211/50261/0/0/139...,http://img4.zergnet.com/1642211_300.jpg,zergnet,http://tmz.com,fab1d021d6f6a57c778988a90aa7803375e2550a.jpg,2017-04-23T06:30:59.835Z,http://www.zergnet.com/news/1642211/magic-john...
2,ObjectId(58fc4a639e1e2a693fe54749),Anthony Bourdain Relieved to No Longer Pretend...,http://www.zergnet.com/i/1597599/50261/0/0/139...,http://img4.zergnet.com/1597599_300.jpg,zergnet,http://tmz.com,db6f8f33bfca04150222fb771cfc86fd572f6e9b.jpg,2017-04-23T06:30:59.889Z,http://www.zergnet.com/news/1597599/anthony-bo...
3,ObjectId(58fc4a639e1e2a693fe5474a),The Real Reason Demi Moore & Ashton Kutcher Go...,http://www.zergnet.com/i/1611238/50261/0/0/139...,http://img5.zergnet.com/1611238_300.jpg,zergnet,http://tmz.com,ba5f7e095f8d048dfbe36997a122205ed8847bcd.jpg,2017-04-23T06:30:59.943Z,http://www.zergnet.com/news/1611238/the-real-r...
4,ObjectId(58fc4a639e1e2a693fe5474b),Carmelo and La La Anthony's Prenup Will Contro...,http://www.zergnet.com/io/1646212/50262/0/0/13...,http://img1.zergnet.com/1646212_300.jpg,zergnet,http://tmz.com,2c2eded673d1577bb7f5e139867bf7d08289eeae.jpg,2017-04-23T06:31:00.003Z,http://www.tmz.com/2017/04/20/carmelo-la-la-an...


In [8]:
data['headline']

0        Why People At NBC Are Reportedly Worried About...
1        Magic Johnson Opens Up About His Son EJ Coming...
2        Anthony Bourdain Relieved to No Longer Pretend...
3        The Real Reason Demi Moore & Ashton Kutcher Go...
4        Carmelo and La La Anthony's Prenup Will Contro...
5        Steve Harvey Comes to the Defense of President...
6           Richard Simmons Goes Out in Public in Disguise
7           Drake's House Burglarized By Oddly Thirsty Fan
8        Magic Johnson Opens Up About His Son EJ Coming...
9        'The Situation' Could Face Up To 20 Years in P...
10       Why Gwen Stefani Went Pale When Blake Announce...
11             Alessandra Ambrosio Stuns at Private Dinner
12                Olivia Munn Arrives Braless at Vancouver
13                Chrissy Teigen Shows Off Her Killer Legs
14                   Nina Dobrev Goes Braless at Coachella
15            Brad Pitt’s Not-Yet Girlfriend Sienna Miller
16       Lourdes Leon Bares Her Bum During Miami Sights.

OK, that's better.

In [9]:
data.describe()

,_id,headline,link,img,provider,source,img_file,date,final_link
count,64147,64147,64147,64147,64147,64147,63975,64147,64147
unique,64147,10260,19212,12744,3,17,12709,64147,16817
top,ObjectId(593aef909e1e2a60560529f8),Here’s Why Guys Are Obsessed With This Underwear…,https://grizly.com/lifestyle/guy-turned-backya...,http://cdn.taboolasyndication.com/libtrc/stati...,taboola,http://tmz.com,db07ff3401037653d665822c5a78617464fe4ef8.jpg,2017-05-18T11:46:26.125Z,https://grizly.com/lifestyle/guy-turned-backya...
freq,1,618,334,398,33577,14436,398,1,334


Already we can see some interesting trends here. Out of 64147 unique records, only 10260 of the headlines are unique, but 19212 of the links are unique and 12709 of the image files are unique (assuming for sure that there were issues with downloading images, but more on that later). So it seems already that there are content links which might reuse the same headline, or image for different destination articles. Let's try to dig into that.

In [12]:
data.summary

AttributeError: 'DataFrame' object has no attribute 'summary'

In [14]:
data.duplicated(subset=['headline', 'link', 'img', 'provider', 'source', 'img_file', 'final_link'])

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
64117    False
64118    False
64119    False
64120     True
64121     True
64122     True
64123    False
64124     True
64125     True
64126     True
64127     True
64128    False
64129     True
64130     True
64131     True
64132     True
64133    False
64134     True
64135     True
64136    False
64137    False
64138     True
64139     True
64140     True
64141     True
64142     True
64143     True
64144     True
64145     True
64146     True
dtype: bool